# Importing Libraries/ Packages

In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, UpSampling2D, Rescaling
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
from tensorflow.keras.applications import MobileNet

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Loading Dataset 

In [ ]:
! wget https://briankeng.com/files/hotdog.tar.gz
! tar -xzf hotdog.tar.gz

--2022-03-29 05:25:27--  https://briankeng.com/files/hotdog.tar.gz
Resolving briankeng.com (briankeng.com)... 192.0.78.240, 192.0.78.156
Connecting to briankeng.com (briankeng.com)|192.0.78.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46732258 (45M) [application/octet-stream]
Saving to: ‘hotdog.tar.gz.3’

hotdog.tar.gz.3     100%[===================>]  44.57M  22.1MB/s    in 2.0s    

2022-03-29 05:25:29 (22.1 MB/s) - ‘hotdog.tar.gz.3’ saved [46732258/46732258]



# Self-Defined Functions for Repeat Use

In [ ]:
def evaluate_model(model, runs=5):
    scores = [] 
    for i in range(runs):
        print('Executing run %d' % (i+1))
        model.fit_generator(train_generator,
                            callbacks=[],
                            steps_per_epoch=num_train_samples // batch_size,
                            epochs=epochs, verbose=0)
        print(' * Evaluating model on test set')
        scores.append(model.evaluate_generator(test_generator, 
                                               steps=num_test_samples // batch_size,
                                               verbose=0))
        print(' * Test set Loss: %.4f, Accuracy: %.4f' % (scores[-1][0], scores[-1][1]))
        
    accuracies = [score[1] for score in scores]     
    return np.mean(accuracies), np.std(accuracies)

# Data Pre-Processing

In [ ]:
img_width, img_height = 150, 150

train_data_dir = 'hotdog/train'
test_data_dir = 'hotdog/test'

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# Part (a): Simple CNN Model

In [ ]:
def model_a():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], 
                  optimizer='rmsprop')
    
    return model

In [ ]:
model_a().summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 activation_15 (Activation)  (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 72, 72, 32)        9248      
                                                                 
 activation_16 (Activation)  (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                 

In [ ]:
batch_size = 50
epochs = 10
train_datagen = ImageDataGenerator(rescale=1. / 255) # rescaling pixels to be between [0, 1]
test_datagen = ImageDataGenerator(rescale=1. / 255) # rescaling pixels to be between [0, 1]

# Data parameters
num_train_samples = 498
num_test_samples = 500

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height), # resizing images to be in img_width x img_height (150x150)
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height), # resizing images to be in img_width x img_height (150x150)
    batch_size=batch_size,
    class_mode='binary'
)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [ ]:
mean_accuracy, std_accuracy = evaluate_model(model=model_a(), runs=5)

Executing run 1
 * Evaluating model on test set
 * Test set Loss: 0.6892, Accuracy: 0.5580
Executing run 2
 * Evaluating model on test set
 * Test set Loss: 1.1645, Accuracy: 0.5700
Executing run 3
 * Evaluating model on test set
 * Test set Loss: 1.5719, Accuracy: 0.5500
Executing run 4
 * Evaluating model on test set
 * Test set Loss: 2.3114, Accuracy: 0.5660
Executing run 5
 * Evaluating model on test set
 * Test set Loss: 3.1927, Accuracy: 0.5720


In [ ]:
print('Mean test set accuracy over 5 runs: %.4f +/- %.4f' % (mean_accuracy, std_accuracy))

Mean test set accuracy over 5 runs: 0.5632 +/- 0.0082


# Part (b): Modeling Using Transfer Learning

In [ ]:
def model_b():
    ''' 
    Uses a base-model (Xception) with pre-trained weights (on ImageNet), scales the input pixels between -1 to 1, and adds a GlobalAveragePooling2D layer.
    '''
    base_model = keras.applications.Xception(
      weights='imagenet',
      input_shape=input_shape,
      include_top=False) # Xception architecture with weights pre-trained on ImageNet
    
    base_model.trainable = False # the layers of the base model are kept frozen

    inputs = keras.Input(shape=input_shape)

    scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1) # pixels are scaled from -1 to 1
    x = scale_layer(inputs)

    x = base_model(x, training=False) # scaling layer is a non-training layer
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)

    outputs = keras.layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs)

    model.compile(loss='binary_crossentropy', metrics=['accuracy'], 
                  optimizer='rmsprop')
    
    return model

In [ ]:
model_b().summary()

83697664/83683744 [==============================] - 3s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 150, 150, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_12 (Dense)            (None, 64)                131136    
                                                                 
 dropout_6 (Dropout)         (None, 64)                0      

In [ ]:
batch_size = 32
epochs = 10
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Data parameters
num_train_samples = 498
num_test_samples = 500

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height), # resizing images to be in img_width x img_height (150x150)
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height), # resizing images to be in img_width x img_height (150x150)
    batch_size=batch_size,
    class_mode='binary'
)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [ ]:
mean_accuracy, std_accuracy = evaluate_model(model=model_b(), runs=5)

Executing run 1
 * Evaluating model on test set
 * Test set Loss: 0.3589, Accuracy: 0.8833
Executing run 2
 * Evaluating model on test set
 * Test set Loss: 0.5333, Accuracy: 0.9062
Executing run 3
 * Evaluating model on test set
 * Test set Loss: 1.0761, Accuracy: 0.8708
Executing run 4
 * Evaluating model on test set
 * Test set Loss: 0.9277, Accuracy: 0.8917
Executing run 5
 * Evaluating model on test set
 * Test set Loss: 1.1651, Accuracy: 0.8917


In [ ]:
print('Mean test set accuracy over 5 runs: %.4f +/- %.4f' % (mean_accuracy, std_accuracy))

Mean test set accuracy over 5 runs: 0.8887 +/- 0.0116
